In [1]:
import os
import pathlib
import shutil
import json
import yaml
import locale
import argparse

import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

locale.getpreferredencoding = lambda: "UTF-8"

#!pip install --upgrade gdown
#clear_output()

In [2]:
# Download the git repository

#!git clone https://github.com/WongKinYiu/yolov9
%cd yolov9
#!pip install -r requirements.txt -q

c:\Mansura\UTI-Revision2\ExternalValidation\yolov9


In [3]:
# @title Model training config
dataset_path = "C:/Mansura/UTI-Revision2/ExternalValidation/A-clinical-1/data-directory"           # @param {type:"string"}
experiment_name = "ev-training.1"              # @param {type:"string"}
model_name = "yolov9-e"  # @param ["yolov9-c", "yolov9-e"]
result_dir = "C:/Mansura/UTI-Revision2/ExternalValidation/EV-YOLOv9e" # @param {type:"string"}
num_epochs = 100        # @param {type:"integer"}
epoch_patience = 100     # @param{type: "integer"}
optimizer = "SGD"       # @param ["SGD", "Adam", "AdamW", "LION"]
image_size = 640        # @param{type: "integer"}]
#learning_rate = 0.01    # @param {type: "number"}
batch_size = 8         # @param {type: "integer"}
conf_threshold = 0.001  # @param {type: "number"}
iou_threshold = 0.5     # @param {type: "number"}
pretrained = True       # @param {type:"boolean"}

weight_name = f"{model_name}.pt"
cfg_name = f"models/detect/{model_name}.yaml"

In [4]:
# Setup dataset configuration

data_config = {
    "path": dataset_path,
    "train": "train",       # training data folder name
    "val": "val",           # validation data folder name
    "test": "test",         # testing data folder name

    # label to class name mappings
    "names": {
        0: "epith",
        1: "rbc/wbc"
    }
}

with open("data-ev.yaml", "w") as f:
    yaml.dump(data_config, f)

In [5]:
# Start training

%run train_dual.py \
--batch-size {batch_size} \
--epochs {num_epochs} \
--img {image_size} \
--device 0 \
--min-items 0 \
--data data-ev.yaml \
--weights "C:/Mansura/UTI-Revision2/ExternalValidation/YOLOv9e/training.1/weights/best.pt" \
--cfg models/detect/{model_name}.yaml \
--hyp hyp.scratch-high.yaml \
--optimizer {optimizer} \
--project {result_dir} \
--name {experiment_name} \
--patience {epoch_patience} \
--exist-ok 


train_dual: weights=C:/Mansura/UTI-Revision2/ExternalValidation/YOLOv9e/training.1/weights/best.pt, cfg=models/detect/yolov9-e.yaml, data=data-ev.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=C:/Mansura/UTI-Revision2/ExternalValidation/EV-YOLOv9e, name=ev-training.1, exist_ok=True, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO  v0.1-104-g5b1ea9a Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_mom

In [9]:
# Test and generate predictions

!python val_dual.py --weights "C:/Mansura/UTI-Revision2/ExternalValidation/EV-YOLOv9e/ev-training.1/weights/best.pt" \
--data data-ev.yaml \
--save-txt \
--save-conf \
--imgsz {image_size} \
--batch {batch_size} \
--conf 0.01 \
--iou {iou_threshold} \
--device 0 \
--project {result_dir} \
--name {"ev-testing.1"} \
--task test \
--exist-ok


val_dual: data=data-ev.yaml, weights=['C:/Mansura/UTI-Revision2/ExternalValidation/EV-YOLOv9e/ev-training.1/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.01, iou_thres=0.5, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=C:/Mansura/UTI-Revision2/ExternalValidation/EV-YOLOv9e, name=ev-testing.1, exist_ok=True, half=False, dnn=False, min_items=0
WARNING  confidence threshold 0.01 > 0.001 produces invalid results
YOLO  v0.1-104-g5b1ea9a Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
yolov9-e summary: 839 layers, 68549356 parameters, 0 gradients, 240.7 GFLOPs

test: Scanning C:\Mansura\UTI-Revision2\ExternalValidation\A-clinical-1\data-directory\test\labels.cache... 210 images, 16 backgrounds, 0 corrupt: 100%|██████████| 210/210 00:00
test: Scanning C:\Mansura\UTI-Revision2\ExternalValidation\A-clinical-1\data-dire